# Imports

##### General imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import networkx as nx

##### Import from flatland environment 

In [3]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator
from flatland.utils.rendertools import RenderTool

##### Import from our framework

In [4]:
from src.graph import NetworkGraph
from src.flows import TimeNetwork
from src.flows import MCFlow

# Test of time expanded network

##### Create a flatland network

In [5]:
number_agents = 10

size_side = 20
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=complex_rail_generator(nr_start_goal=20, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, 
                                                    seed = np.random.randint(0,2000)),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_agents,
              obs_builder_object=GlobalObsForRailEnv())

env.reset()

env_renderer = RenderTool(env)
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)


matrix_rail = np.array(env.rail.grid.tolist())
flatlandNetwork = NetworkGraph(matrix_rail,[(0,1)],[(1,0)])

cycles detected in NetworkGraph, this may be due to the fact that endpoint can be used to do 180 turn


In [6]:
sources = []
sinks = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)

##### create a time expanded network

In [7]:
import time

In [8]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=50)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

time taken to build the graph: 10.841176271438599


In [9]:
TestNetworkTime.connect_sources_and_sink(sources,sinks)

##### Test LP Formulation

test a simple graph

In [10]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),TestNetworkTime.topology)

Academic license - for non-commercial use only


In [11]:
mcflow.solve()

Optimize a model with 755880 rows, 1132600 columns and 4252800 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 750618 rows and 1122118 columns
Presolve time: 1.89s
Presolved: 5262 rows, 10482 columns, 21778 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.215e+04
 Factor NZ  : 9.910e+04 (roughly 7 MBytes of memory)
 Factor Ops : 2.457e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.82212961e+05 -3.13366095e+05  6.63e+02 1.17e+01  4.72e+02     2s
   1   5.05558780e+04 -3.02228898e+05  1.81e+02 5.68e-14  1.34e+02     2s
   2   1.38559547e+04 -2.18385135e+05  4.71e+01 8.17e-14  3.74e+01     2s
   3   2.95350226e+03 

In [12]:
paths = mcflow.extract_paths()

In [13]:
mcflow.check_no_collisions_solution(paths)

True

In [14]:
paths

{0: ['source_agent_0',
  '(12, 7)_S_in_t1',
  '(12, 7)_N_out_t2',
  '(11, 7)_S_in_t3',
  '(11, 7)_N_out_t4',
  '(10, 7)_S_in_t5',
  '(10, 7)_N_out_t6',
  '(9, 7)_S_in_t7',
  '(9, 7)_N_out_t8',
  '(8, 7)_S_in_t9',
  '(8, 7)_W_out_t10',
  '(8, 6)_E_in_t11',
  '(8, 6)_N_out_t12',
  '(7, 6)_S_in_t13',
  '(7, 6)_W_out_t14',
  '(7, 5)_E_in_t15',
  '(7, 5)_N_out_t16',
  '(6, 5)_S_in_t17',
  '(6, 5)_W_out_t18',
  '(6, 4)_E_in_t19',
  '(6, 4)_N_out_t20',
  '(5, 4)_S_in_t21',
  '(5, 4)_W_out_t22',
  '(5, 3)_E_in_t23',
  '(5, 3)_N_out_t24',
  '(4, 3)_S_in_t25',
  '(4, 3)_W_out_t26',
  '(4, 2)_E_in_t27',
  '(4, 2)_N_out_t28',
  '(3, 2)_S_in_t29',
  '(3, 2)_W_out_t30',
  '(3, 1)_E_in_t31',
  '(3, 1)_N_out_t32',
  '(2, 1)_S_in_t33',
  '(2, 1)_W_out_t34',
  '(2, 0)_E_in_t35',
  '(2, 0)_N_out_t36',
  '(1, 0)_S_in_t37',
  'sink_agent_0'],
 1: ['source_agent_1',
  '(14, 18)_E_in_t1',
  '(14, 18)_W_out_t2',
  '(14, 17)_E_in_t3',
  '(14, 17)_W_out_t4',
  '(14, 16)_E_in_t5',
  '(14, 16)_W_out_t6',
  '(14, 

In [15]:
paths_simple = ([x.split("_")[0] for x in paths[0]])

In [16]:
paths_simple

['source',
 '(12, 7)',
 '(12, 7)',
 '(11, 7)',
 '(11, 7)',
 '(10, 7)',
 '(10, 7)',
 '(9, 7)',
 '(9, 7)',
 '(8, 7)',
 '(8, 7)',
 '(8, 6)',
 '(8, 6)',
 '(7, 6)',
 '(7, 6)',
 '(7, 5)',
 '(7, 5)',
 '(6, 5)',
 '(6, 5)',
 '(6, 4)',
 '(6, 4)',
 '(5, 4)',
 '(5, 4)',
 '(5, 3)',
 '(5, 3)',
 '(4, 3)',
 '(4, 3)',
 '(4, 2)',
 '(4, 2)',
 '(3, 2)',
 '(3, 2)',
 '(3, 1)',
 '(3, 1)',
 '(2, 1)',
 '(2, 1)',
 '(2, 0)',
 '(2, 0)',
 '(1, 0)',
 'sink']